# 1

## Implementación de librerias

In [1]:
import numpy as np
from PIL import Image

# 1.1

## Funciones de descretizacion

In [ ]:
import numpy as np
from PIL import Image

# Funcion para determinar si es un color cercano a
def colorSimilar(color, referencia, tolerancia=20):
    return all(abs(c1 - c2) <= tolerancia for c1, c2 in zip(color, referencia))

# Función para clasificar una celda según el color predominante
def clasificarCelda(pixeles):
    conteo = {"0": 0, "1": 0, "S": 0, "G": 0}

    BLANCO = (255, 255, 255)
    NEGRO = (0, 0, 0)
    ROJO = (255, 0, 0)
    VERDE = (0, 255, 0)

    for fila in pixeles:
        for r, g, b in fila:
            if colorSimilar((r, g, b), BLANCO):
                conteo["0"] += 1
            elif colorSimilar((r, g, b), NEGRO):
                conteo["1"] += 1
            elif colorSimilar((r, g, b), ROJO):
                conteo["S"] += 1
            elif colorSimilar((r, g, b), VERDE):
                conteo["G"] += 1

    return max(conteo, key=conteo.get)

def discretizar_imagen(ruta_imagen, tamano_celda):
    imagen = Image.open(ruta_imagen).convert("RGB")
    pixeles = np.array(imagen)
    alto, ancho, _ = pixeles.shape

    filas = alto // tamano_celda
    columnas = ancho // tamano_celda

    laberinto = np.zeros((filas, columnas), dtype=str)

    for y in range(filas):
        for x in range(columnas):
            celda_pixeles = pixeles[y * tamano_celda:(y + 1) * tamano_celda, 
                                    x * tamano_celda:(x + 1) * tamano_celda]

            laberinto[y, x] = clasificarCelda(celda_pixeles)

    return laberinto



Matriz del laberinto:
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 G G G 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 G G G 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 G G G 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

## Aplicacion

In [ ]:
# Imagen de prueba
ruta = "img/Prueba Lab1.bmp"

# Tamaño de pixel definido
tamano_celda = 10  

# Convertir la imagen en una matriz de celdas
matriz_laberinto = discretizar_imagen(ruta, tamano_celda)

# Mostrar la matriz discretizada
print("\nMatriz del laberinto:")
for fila in matriz_laberinto:
    print(" ".join(fila))